In [1]:
%%shell
FILE=/content/data/PASCALVOC2007.zip
if [ ! -f "$FILE" ]; then
    mkdir /content/data
    wget  https://data.deepai.org/PASCALVOC2007.zip -P /content/data
    unzip $FILE -d /content/data
fi

In [2]:
# -*- coding: utf-8 -*-
from utils.agent import *
from utils.dataset import *
from config import *

from IPython.display import clear_output

import sys
import traceback
import sys
import os
import tqdm as tq
import seaborn as sns

#!pip3 install torch==1.5.1 torchvision==0.6.1 -f https://download.pytorch.org/whl/cu92/torch_stable.html

# train_loader2012, val_loader2012 = read_voc_dataset(download=LOAD, year='2012')
train_loader2007, val_loader2007 = read_voc_dataset(download=LOAD, year='2007')

### Séparation des différents éléments du jeu de données selon la classe.

In [3]:
classes = [ 'cat', 'dog', 'bird', 'motorbike','diningtable', 'train', 'tvmonitor', 'bus', 'horse', 'car', 'pottedplant', 'person', 'chair', 'boat', 'bottle', 'bicycle', 'aeroplane', 'cow', 'sheep', 'sofa']

agents_per_class = {}
datasets_per_class = sort_class_extract([train_loader2007])
datasets_eval_per_class = sort_class_extract([val_loader2007])

### Entraînement sur chaque classe.

In [ ]:
def predict_image(agent, datasets_per_class, classe):
    for i in range(20):
        index = random.choice(list(datasets_per_class[classe].keys()))
        image, gt_boxes = extract(index, datasets_per_class[classe])
        agent.predict_image(image, plot=True)

In [ ]:
for i in range(len(classes)):
    classe = classes[i]
    print(f"Classe {i} :"+str(classe)+"...")
    agents_per_class[classe] = Agent(classe, alpha=0.2, num_episodes=1, load=False)
    agents_per_class[classe].train(datasets_per_class[classe])
    del agents_per_class[classe]
    # torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()
results = {}
for i in classes:
    results[i] = []

for i in tq.tqdm(range(len(classes))):
    classe = classes[i]
    print("Class "+str(classe)+"...")
    agent = Agent(classe, alpha=0.2, num_episodes=15, load=True)
    res = agent.evaluate(datasets_per_class[classe])
    results[classe] = res